In [160]:
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
# PyTorch
from torchvision import transforms, models
import torch
from torch import optim, cuda
import torch.nn as nn
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os
# Useful for examining network
from torchsummary import summary
# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt

In [40]:
# Location of data
datadir = './Dataset/MTCNNAlignedCroppedFaces/'
traindir = datadir + 'train/'
validdir = datadir + 'val/'
testdir = datadir + 'test/'
save_file_name = 'face_selection-1039.pt'
checkpoint_path = 'face_selection-1039.pth'

# Change to fit hardware
batch_size = 8

# Whether to train on a gpu
train_on_gpu = cuda.is_available()
#train_on_gpu = False
print(f'Train on gpu: {train_on_gpu}')

# Number of gpus
if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'{gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False

Train on gpu: False


In [41]:
image_datasets = {x : (os.path.join(datadir, x))
                    for x in ['train', 'val']}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [42]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']
#test_dataset = image_datasets['test']

pri_train = sorted(os.listdir(datadir + 'train/private/'))
pub_train = sorted(os.listdir(datadir + 'train/public/'))
semi_train = sorted(os.listdir(datadir + 'train/semiprivate/'))

pri_val = sorted(os.listdir(datadir + 'val/private/'))
pub_val = sorted(os.listdir(datadir + 'val/public/'))
semi_val = sorted(os.listdir(datadir + 'val/semiprivate/'))

#pri_test = sorted(os.listdir(datadir + 'test/private/'))
#pub_test = sorted(os.listdir(datadir + 'test/public/'))
#semi_test = sorted(os.listdir(datadir + 'test/semiprivate/'))
pri_train_filelist = [x.split('.')[0] for x in pri_train]
pub_train_filelist = [x.split('.')[0] for x in pub_train]
semi_train_filelist = [x.split('.')[0] for x in semi_train]

pri_val_filelist = [x.split('.')[0] for x in pri_val]
pub_val_filelist = [x.split('.')[0] for x in pub_val]
semi_val_filelist = [x.split('.')[0] for x in semi_val]

#pri_test_filelist = [x.split('.')[0] for x in pri_test]
#pub_test_filelist = [x.split('.')[0] for x in pub_test]
#semi_test_filelist = [x.split('.')[0] for x in semi_test]
print(pri_train_filelist[:5])
print(pub_train_filelist[:5])
print(semi_train_filelist[:5])

print(pri_val_filelist[:5])
print(pub_val_filelist[:5])
print(semi_val_filelist[:5])

#print(pri_test_filelist[:5])
#print(pub_test_filelist[:5])
#print(semi_test_filelist[:5])
train_filelist = pri_train_filelist  + pub_train_filelist + semi_train_filelist
val_filelist = pri_val_filelist  + pub_val_filelist + semi_val_filelist
#test_filelist = pri_test_filelist  + pub_test_filelist + semi_test_filelist

['00000ae5e7fcc87222f1fb6f221e7501558e5b08_0', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_1', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_2', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_3', '00000ae5e7fcc87222f1fb6f221e7501558e5b08_4']
['00bc0b7df6951edbe08ac69cfb00f0b7a9927a31_0', '00bc0b7df6951edbe08ac69cfb00f0b7a9927a31_1', '00bc0b7df6951edbe08ac69cfb00f0b7a9927a31_2', '00bde3f3037497aaa1e665ba1b23fc149ec9081f_0', '00c5d8c5b83e61cfcf1bd0dffb0911230d25ce7b_0']
['00ce1cf572bcf97c138129087291ba7cce7ed8b3_0', '00ce1cf572bcf97c138129087291ba7cce7ed8b3_1', '00ce1cf572bcf97c138129087291ba7cce7ed8b3_2', '00ce1cf572bcf97c138129087291ba7cce7ed8b3_3', '0339520eef09650d55879247cba49c752a2c0b1e_0']
['04085231fc015457761e587ee53e4d1be6eb3666_0', '04085231fc015457761e587ee53e4d1be6eb3666_1', '04085231fc015457761e587ee53e4d1be6eb3666_2', '04085231fc015457761e587ee53e4d1be6eb3666_3', '11708847396_9383898125_b_0']
['052e744bb87e87adebaf05d4da1fea43c91519be_0', '052e744bb87e87adebaf05d4da1fea43c91519be_1',

In [43]:
coor_dir = './Dataset/FaceCoordinates/'
coor_traindir = coor_dir + 'train/'
coor_validdir = coor_dir + 'val/'
coor_testdir = coor_dir + 'test/'

fea_dir = './Dataset/FaceFeature/'
fea_traindir = fea_dir + 'train/'
fea_validdir = fea_dir + 'val/'
fea_testdir = fea_dir + 'test/'

face_dir = './Dataset/MTCNNAlignedCroppedFaces/'
face_traindir = face_dir + 'train/'
face_validdir = face_dir + 'val/'
face_testdir = face_dir + 'test/'

pose_path = './Dataset/inputvector/headpose/'
gaze_path = './Dataset/inputvector/eyegaze/'
blur_path = './Dataset/inputvector/blur/'
position_path = './Dataset/inputvector/faceposition/'

input_path = './Dataset/inputvector/input/'
label_path = './Dataset/output/'

# Network

In [245]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 10)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        #self.fc2 = nn.Linear(10, 20)
        #self.prelu = nn.PReLU(1)
        self.out = nn.Linear(10, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        a1 = self.fc1(input_)
        h1 = self.relu1(a1)
        dout = self.dout(h1)
        #a2 = self.fc2(dout)
        #h2 = self.prelu(a2)
        a3 = self.out(dout)
        y = self.out_act(a3)
        return y
    
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(6, 10),
            nn.ReLU(),
            nn.Linear(10, 2)
        )
        
    def forward(self, x):
        # convert tensor (128, 1, 28, 28) --> (128, 1*28*28)
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x    

net = Net()
net.double()
print(net)  # net architecture

Net(
  (fc1): Linear(in_features=6, out_features=10, bias=True)
  (relu1): ReLU()
  (dout): Dropout(p=0.2)
  (out): Linear(in_features=10, out_features=1, bias=True)
  (out_act): Sigmoid()
)


# Loss function and Optimizer

In [246]:
#criterion = nn.BCELoss(size_average=True)
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss(size_average=False)
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [247]:
mean_train_losses = []
mean_valid_losses = []
valid_acc_list = []
epochs = 50
for epoch in range(epochs):
    net.train()
    
    train_losses = []
    valid_losses = []
    for i, inputs in enumerate(train_filelist):
        inputs = torch.from_numpy(np.load(input_path + 'train/' + train_filelist[i] + '.npy'))
        face_num = int(train_filelist[i].rsplit('_')[-1])
        image_name = train_filelist[i].rsplit('_',1)[-2]
        with open(label_path + 'train/' + image_name + '.txt') as f:
            for j, line in enumerate(f):
                if j == face_num:
                    target = torch.tensor(np.double(line)).double()
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if i % (1000) == 0:
            print(f'{i} / 6500')
            

    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, inputs in enumerate(val_filelist):
            inputs = torch.from_numpy(np.load(input_path + 'val/' + val_filelist[i] + '.npy'))
            face_num = int(val_filelist[i].rsplit('_')[-1])
            image_name = val_filelist[i].rsplit('_',1)[-2]
            with open(label_path + 'val/' + image_name + '.txt') as f:
                for j, line in enumerate(f):
                    if j == face_num:
                        target = torch.tensor(np.double(line)).double()
                    outputs = net(inputs)
                    loss = criterion(outputs, target)
                    valid_losses.append(loss.item())
            
                    predicted = outputs
                    correct += (predicted == target).sum().item()
                    total += target
            
    mean_train_losses.append(np.mean(train_losses))
    mean_valid_losses.append(np.mean(valid_losses))
    
    accuracy = 100*correct/total
    valid_acc_list.append(accuracy)
    print('epoch : {}, train loss : {:.4f}, valid loss : {:.4f}, valid acc : {:.2f}%'\
          .format(epoch+1, np.mean(train_losses), np.mean(valid_losses), accuracy))



0 / 6500
1000 / 6500
2000 / 6500
3000 / 6500
4000 / 6500
5000 / 6500
6000 / 6500
epoch : 1, train loss : 0.4594, valid loss : 0.4291, valid acc : 110.71%
0 / 6500
1000 / 6500
2000 / 6500
3000 / 6500
4000 / 6500
5000 / 6500
6000 / 6500
epoch : 2, train loss : 0.4604, valid loss : 0.4291, valid acc : 110.97%
0 / 6500
1000 / 6500
2000 / 6500
3000 / 6500
4000 / 6500
5000 / 6500
6000 / 6500


KeyboardInterrupt: 